In [3]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re

In [4]:
url = 'https://news.nate.com/view/20200314n09669?mid=n1008'

In [5]:
import urllib.request
def get_text(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = ''
    for item in soup.find_all('div', id='realArtcContents'):
        text = text + str(item.find_all(text=True))
    return text

In [6]:
def clean_text(text):
    cleaned_text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\.]+", " ", str(text))
    return cleaned_text

In [7]:
def get_title(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'html.parser', from_encoding='utf-8')
    text = soup.find('h3',class_='articleSubecjt')
    return text

In [8]:
wd = "C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver.exe"

In [9]:
def comment(url):
    driver = webdriver.Chrome(wd)
    url_nums = re.findall("\d+", url)
    addr = 'http://comm.news.nate.com/Comment/ArticleComment/List?artc_sq='+url_nums[0]+'n'+url_nums[1] # 댓글창 주소
    driver.get(addr)
    pages = 2
    try:
        for i in range(5):
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"body > div.commentBox.reply_hide > div:nth-child(3) > div.paging_wrap > a:nth-child("+str(pages)+")"))).click()
            time.sleep(2)
            print(pages, end=" ")
            pages+=1
    except exceptions.ElementNotVisibleException as e: # 페이지 끝
        pass
    except Exception as e: # 다른 예외 발생시 확인
        print(e)
    html = driver.page_source
    dom = BeautifulSoup(html, "lxml")
    # 댓글이 들어있는 페이지 전체 크롤링
    comments_raw = dom.find_all('dd',{'class':'usertxt'})
    # 댓글의 text만 뽑는다.
    comments = [comment.text.strip() for comment in comments_raw]
    return comments

In [10]:
s = comment(url)
s

2 3 4 5 6 

['솔직히 급 노화...\n아저씨 다됐음ㅋ',
 '너무 예민한것 아닌지.',
 '유승호를 외모로 까?ㅋㅋㅋ',
 '승호야ㅋㅋㅋㅋㅋㅋ 한귀로 듣고 흘릴 가치도 없다 걍 무시해ㅋㅋㅋ',
 '살찐것도 모르겠고 잘생기기만 했어',
 '어디가 살쪘다는건지 알수가 없네요? 훈남인데 별 악플이다있네요 드라마 잼있구 본방사수중이에요~^^',
 '유승호가 100kg여도 나보다는 잘생겼을거같은데;',
 '근데 유승호도 연기가 참안늘어  연기에 깊이가 없다고 할까?',
 '어디가 쪘다는 건지? 잘생겼는뎅~~',
 '아니 누가 누굴보고 외모 지적을해?\n남자가 봐도 존잘인데 뭐지;;',
 '외모고 자시고 연기도 별로고 드라마가 너무 재미없더라ㅋㅋㅋ',
 '오징어들이 누구를 건드나',
 '우리나라에서 제일 잘생긴 배우 중 하나라 생각했는데 유승호도 외모악플이 신경쓰이는게 신기하다... 연기잔데 배역따라서 살 좀 찌울 수 있지',
 '악플러들 불러다가 같이 셀카찍어 인스타 올려라 승호야 ㅋㅋㅋ',
 '잘생겼구만  지적한사람면상보자',
 '유승호 내이상형 ㅠㅠ',
 '아니 유승호가 외모지적으로 까여?ㅋㅋㅋ  존잘이던데',
 '드라마 안보는데 유승호 때문에 메모리스트 봄. 어디가 살이찜? 진짜 소름끼칠 정도로 잘생겼던데.',
 '나이들수록 얼굴에 살이 있어야 건강해보인단다',
 '그살 나한테로와 찐거니\n대체 어디가 살찐거라는...']

In [11]:
t = get_text(url)
cl = clean_text(t)
print(cl)

 톱스타뉴스 김효진 메모리스트 유승호가 외모 악플에 대한 심경을 고백한 후 인스타그램을 업데이트했다. 앞서 지난 일 유승호는 자신의 인스타스토리를 통해 경찰 역할이라 일부러 살 많이 찌웠어요. 저도 알아요 얼굴 살찐 거 라는 글을 남긴 바 있다. 일부 네티즌의 악플에 대다수의 네티즌은 잘생겼는데 어디가 찐거냐 며 이해가 안된다는 반응을 보였다. 자신의 글이 이슈가 되자 유승호는 결국 인스타스토리를 삭제했고 메모리스트 촬영 사진을 남기며 인스타그램 활동을 이어가고 있다. 일 오후 유승호는 자신의 인스타그램에 웃자 라는 글과 함께 사진 한 장을 게재했다. 공개된 사진 속에는 경찰 제복을 입고 메모리스트 를 촬영 중인 유승호의 모습이 담겨 있다. 특히 유승호의 환한 미소가 훈훈함을 자아낸다. 극중 유승호는 사이코메트리 초능력을 가진 형사이자 전 세계 유일무이 공인된 초능력자 동백 역을 맡아 열연 중에 있다. 한편 유승호 이세영 등이 출연 중인 메모리스트 는 매주 수 목요일 오후 시 분 방송된다. 김효진 . . 취재 및 보도 . . 톱스타뉴스 . . 무단전재 재배포 금지 대한민국 . 스타 사진뉴스 . 사진 을 클릭하면 국내 최대 사이즈인 사진을 볼 수 있습니다. 메모리스트 유승호 붉은 돼지 발견했다 재방송 넷플릭스 방영 여부는 메모리스트 유승호 검찰 허수아비 때려눕힌 맨손 액션 통쾌 직캠 유승호 년 올해는 유승호가 대세 유승호 외모 악플에 심경 고백 인스타 글 삭제 네티즌 무분별 비난 그만 지적 메모리스트 유승호 이세영 회 예고 공개 뜨거운 시청자 반응 . . . . . . . . . . . 


In [12]:
from krwordrank.sentence import summarize_with_sentences
def keyword_sentence(url):#키워드 문장 추출
    m_text = get_text(url)
    r_text = clean_text(m_text)
    data = r_text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=1, num_keysents=10)
    keyword = list(keywords.keys())
    return keyword[0], sents

In [13]:
keyword, sents = keyword_sentence(url)
sents

[' 톱스타뉴스 김효진 메모리스트 유승호가 외모 악플에 대한 심경을 고백한 후 인스타그램을 업데이트했다',
 ' 저도 알아요 얼굴 살찐 거 라는 글을 남긴 바 있다',
 ' 일부 네티즌의 악플에 대다수의 네티즌은 잘생겼는데 어디가 찐거냐 며 이해가 안된다는 반응을 보였다',
 ' 김효진 ',
 ' ',
 ' 취재 및 보도 ',
 ' ',
 ' 톱스타뉴스 ',
 ' ',
 ' 무단전재 재배포 금지 대한민국 ']

In [14]:
keyword

'유승호'

In [15]:
from tensorflow.keras.models import load_model
from konlpy.tag import Okt  
okt = Okt()

In [16]:
from selenium import webdriver
from selenium.common import exceptions
import re
import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
def model_load(comments):
    token_sent=[]
    for comment in comments:
        temp = okt.morphs(comment, stem=True)
        token_sent.append(temp)
        max_words = 35000
    tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
    tokenizer.fit_on_texts(token_sent) 
    token_sent = tokenizer.texts_to_sequences(token_sent)
    word_to_index = tokenizer.word_index
    vocab_size = len(word_to_index)+1
    max_len = 124
    X_data = pad_sequences(token_sent, maxlen=max_len)
    model = load_model('model8n.h5')
    predict = model.predict_classes(X_data)
    #for i in range(len(X_data)):
    #    if(predict[i] == 0):
    #        a=0
    return predict

In [18]:
s[0]

'솔직히 급 노화...\n아저씨 다됐음ㅋ'

In [19]:
print(s[1])

너무 예민한것 아닌지.


In [20]:
model_result = model_load(s)

In [21]:
print(model_result)
print(np.delete(model_result,1))
model_result.dtype

[[1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]]
[1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0]


dtype('int32')

In [22]:
count = 0
number = []
for i in range(1,len(model_result)):
    if model_result[i] == 1:
        count += 1
        number.append(i)
print(count)
print(number)

5
[1, 4, 6, 17, 18]


In [23]:
def count(num):
    count = 0
    number = []
    for i in range(1, len(num)):
        if num[i] == 1:
            count += 1
            number.append(i)
    return count, number

In [24]:
a,b = count(model_result)
print(a)
print(b)

5
[1, 4, 6, 17, 18]


In [25]:
actor = ["요즘 연기 정말 잘보고있어요! 항상 응원해요",
        "항상 열심히 하는 모습 너무 보기 좋아요:)",
        "최근 작품에서 연기 정말 많이 늘었더라고요!",
        "연기가 너무 좋아서 응원해요~",
        "요즘 내가 가장 좋아하는 배우 중 한 명ㅎㅎ",
        "어떤 연기를 해도 보는 사람의 심금을 울리는 배우",
        "요즘 얼굴 정말 좋아보여요",
        "예전부터 지금까지 계속해서 응원하고 있어요:)",
        "연기로는 절대 뭐라 못하는 배우",
        "연기신",
        "정말 얼굴 천재..그 자체",
        "요즘 너무 좋아요!",
        "진짜 송강호급 연기자다 연기레알 개잘함.",
        "연기 너무 잘해ㅠㅠㅠㅠ",
        "이번년도 연기 대상 주자!!!!!!",
        "발성.발음.표정...참 몰입되는 배우님...응원합니다.",
        "대상 받아야해요....",
        "난 옛날부터연기잘하는거알고있었지.. ",
        "사랑해요!!!!!!!!!!!",
        "걍 지린다.. 연기 너무 잘하시는 듯",
        "진짜 연기 미쳤어.......",
        "진짜 깔게 없는 연기력 .. ",
        "표정 연기가 너무 섬세하셔서 숨 안쉬고 보게 되는.. 응원합니다 배우님 :>",
        "연기 볼 때마다 놀라요 흡입력이 진짜.. ",
        "진심 시상식에서 상 받아야할 신들린 연기자"]

In [ ]:
singer = ["미세먼지 한톨도 안먹은 목소리의 주인공 진짜.......",
         "음악성은 진짜..말도 안된다고 생각한다.. ",
         "한국 가수 역사상 최고인 사람",
         "목소리가 진짜 보물이야",
         "목소리가 보석같다",
         ""]

In [51]:
import string
import random

def select(text, number):#기사 내용 중 단어 선별하기
    lines = []
    for word in text:
        lines = str.split(text,".")
        for each_line in lines:
            if each_line.find("인스타") > 0:
                print(random.sample(actor, number))

In [52]:
select(cl, a)

['정말 얼굴 천재..그 자체', '예전부터 지금까지 계속해서 응원하고 있어요:)', '어떤 연기를 해도 보는 사람의 심금을 울리는 배우', '연기신', '연기로는 절대 뭐라 못하는 배우']
['연기로는 절대 뭐라 못하는 배우', '연기 볼 때마다 놀라요 흡입력이 진짜.. ', '어떤 연기를 해도 보는 사람의 심금을 울리는 배우', '예전부터 지금까지 계속해서 응원하고 있어요:)', '사랑해요!!!!!!!!!!!']
['연기 너무 잘해ㅠㅠㅠㅠ', '표정 연기가 너무 섬세하셔서 숨 안쉬고 보게 되는.. 응원합니다 배우님 :>', '연기가 너무 좋아서 응원해요~', '연기신', '어떤 연기를 해도 보는 사람의 심금을 울리는 배우']
['어떤 연기를 해도 보는 사람의 심금을 울리는 배우', '걍 지린다.. 연기 너무 잘하시는 듯', '정말 얼굴 천재..그 자체', '연기가 너무 좋아서 응원해요~', '연기로는 절대 뭐라 못하는 배우']
['연기가 너무 좋아서 응원해요~', '연기신', '발성.발음.표정...참 몰입되는 배우님...응원합니다.', '요즘 내가 가장 좋아하는 배우 중 한 명ㅎㅎ', '요즘 연기 정말 잘보고있어요! 항상 응원해요']
['진짜 송강호급 연기자다 연기레알 개잘함.', '요즘 얼굴 정말 좋아보여요', '이번년도 연기 대상 주자!!!!!!', '어떤 연기를 해도 보는 사람의 심금을 울리는 배우', '대상 받아야해요....']
['최근 작품에서 연기 정말 많이 늘었더라고요!', '대상 받아야해요....', '요즘 연기 정말 잘보고있어요! 항상 응원해요', '연기로는 절대 뭐라 못하는 배우', '연기 너무 잘해ㅠㅠㅠㅠ']
['연기 볼 때마다 놀라요 흡입력이 진짜.. ', '대상 받아야해요....', '이번년도 연기 대상 주자!!!!!!', '진짜 연기 미쳤어.......', '연기가 너무 좋아서 응원해요~']
['연기가 너무 좋아서 응원해요~', '정말 얼굴 천재..그 자체', '이번년도 연기 대상 주자!!!!!!', '요즘 얼굴 정말 좋아보여요',

In [23]:
def delete(result, number):
    new = np.delete(result, number)
    return new

In [24]:
def de_comment(comments, number):
    new = np.delete(comments, number)
    return new

In [25]:
stra = de_comment(s, b)
stra_num = delete(model_result, b)
print(stra)
print(stra_num)

['내가 저얼굴이면 누가 외모지적해도 피식도 안할거같은데 맘이 여린가' '솔직히 급 노화...\n아저씨 다됐음ㅋ'
 '어디가 살쪘다는건지 알수가 없네요? 훈남인데 별 악플이다있네요 드라마 잼있구 본방사수중이에요~^^'
 '어디가 쪘다는 건지? 잘생겼는뎅~~' '아니 누가 누굴보고 외모 지적을해?\n남자가 봐도 존잘인데 뭐지;;'
 '외모고 자시고 연기도 별로고 드라마가 너무 재미없더라ㅋㅋㅋ'
 '우리나라에서 제일 잘생긴 배우 중 하나라 생각했는데 유승호도 외모악플이 신경쓰이는게 신기하다... 연기잔데 배역따라서 살 좀 찌울 수 있지'
 '악플러들 불러다가 같이 셀카찍어 인스타 올려라 승호야 ㅋㅋㅋ' '아니 유승호가 외모지적으로 까여?ㅋㅋㅋ  존잘이던데']
[0 0 0 0 0 0 0 0 0]


In [26]:
def count2(num): #count는 악플 개수, number은 악플 위치
    number = []
    for i in range(1, len(num)):
        if num[i] == 0:
            number.append(i)
    return number

In [27]:
def de_comment2(comments, number):
    new = np.delete(comments, number)
    return new

In [28]:
c = count2(model_result)
new = de_comment(s, c)
print(new)

['내가 저얼굴이면 누가 외모지적해도 피식도 안할거같은데 맘이 여린가' '너무 예민한것 아닌지.' '유승호를 외모로 까?ㅋㅋㅋ'
 '승호야ㅋㅋㅋㅋㅋㅋ 한귀로 듣고 흘릴 가치도 없다 걍 무시해ㅋㅋㅋ' '살찐것도 모르겠고 잘생기기만 했어'
 '유승호가 100kg여도 나보다는 잘생겼을거같은데;' '근데 유승호도 연기가 참안늘어  연기에 깊이가 없다고 할까?'
 '오징어들이 누구를 건드나' '잘생겼구만  지적한사람면상보자' '유승호 내이상형 ㅠㅠ'
 '드라마 안보는데 유승호 때문에 메모리스트 봄. 어디가 살이찜? 진짜 소름끼칠 정도로 잘생겼던데.'
 '나이들수록 얼굴에 살이 있어야 건강해보인단다']


In [50]:
import pandas as pd

comment_data = pd.DataFrame(s)
comment_data.columns = ['comment']
comment_data

,comment
0,저런 이목구비 찾아봐.. 없어.. 유승호 잘생겼으니까 헛소리 하지들 말자
1,외모 지적은 류준열이나 김제동같은애들이나 받는거지
2,삼성 이재용 재산걱정하는소리같은소리하고있네\n씅질나게
3,찐것도 모르겠는데 그냥 너무 잘생겼어 우리 승호
4,외모는 문제가 없어 작품 고르는 안목이 문제인거지
5,"하나 분명히 말하는 거지만 실제로 진짜 남자들은 살찌면 어, 살쪘네? 하고 말지 남..."
6,어떻게 저얼굴을...지적해...니들아빠가 원빈급이면..조금이해는해볼게..사진좀......
7,유승호에게 외모악플이란 이재용 재산걱정해주는것과 같음
8,"세상에, 너무 말라서 좀 쪘으면 했는데 살쪘다라니. 유승호 배우님 언제나 응원합니다..."
9,난 지금 모습 좋은데..원래 아역배우들 슬럼프 못이기는데.. 유승호는 아역배우 이미...


In [51]:
in_result = np.insert(model_result, 0, 0)
de_result = np.delete(in_result,0)
print(de_result)

[0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 1 0 1 1 0]


In [52]:
de_data = pd.DataFrame(de_result)
de_data.columns = ['label']
de_data

,label
0,0
1,0
2,0
3,1
4,0
5,0
6,1
7,0
8,0
9,1


In [53]:
df_new = pd.concat([de_data, comment_data], axis = 1)
df_new

,label,comment
0,0,저런 이목구비 찾아봐.. 없어.. 유승호 잘생겼으니까 헛소리 하지들 말자
1,0,외모 지적은 류준열이나 김제동같은애들이나 받는거지
2,0,삼성 이재용 재산걱정하는소리같은소리하고있네\n씅질나게
3,1,찐것도 모르겠는데 그냥 너무 잘생겼어 우리 승호
4,0,외모는 문제가 없어 작품 고르는 안목이 문제인거지
5,0,"하나 분명히 말하는 거지만 실제로 진짜 남자들은 살찌면 어, 살쪘네? 하고 말지 남..."
6,1,어떻게 저얼굴을...지적해...니들아빠가 원빈급이면..조금이해는해볼게..사진좀......
7,0,유승호에게 외모악플이란 이재용 재산걱정해주는것과 같음
8,0,"세상에, 너무 말라서 좀 쪘으면 했는데 살쪘다라니. 유승호 배우님 언제나 응원합니다..."
9,1,난 지금 모습 좋은데..원래 아역배우들 슬럼프 못이기는데.. 유승호는 아역배우 이미...


In [55]:
X_data = df_new['comment']
y_data = df_new['label']

In [60]:
normalized_text = []
for string in X_data.tolist():
    try:
        tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ", string.lower())
    except Exception as e:
        print(string)
        break
    normalized_text.append(tokens)
print(normalized_text[:5])

['저런 이목구비 찾아봐 없어 유승호 잘생겼으니까 헛소리 하지들 말자', '외모 지적은 류준열이나 김제동같은애들이나 받는거지', '삼성 이재용 재산걱정하는소리같은소리하고있네 씅질나게', '찐것도 모르겠는데 그냥 너무 잘생겼어 우리 승호', '외모는 문제가 없어 작품 고르는 안목이 문제인거지']


In [61]:
df_new.comment = normalized_text
for sentence in df_new['comment']:
    print(sentence)

저런 이목구비 찾아봐 없어 유승호 잘생겼으니까 헛소리 하지들 말자
외모 지적은 류준열이나 김제동같은애들이나 받는거지
삼성 이재용 재산걱정하는소리같은소리하고있네 씅질나게
찐것도 모르겠는데 그냥 너무 잘생겼어 우리 승호
외모는 문제가 없어 작품 고르는 안목이 문제인거지
하나 분명히 말하는 거지만 실제로 진짜 남자들은 살찌면 어 살쪘네 하고 말지 남의 까지 쳐들어가서 아가리 털진 않습니다 외모가지고 굳이 쳐들어갈정도로 시간을 투자할 사람들은 아니거든요 다른 이유면 모를까 근데 외모가지고 그정도로 집착하는거면 일베같은거 하는 찐따들이지 여자들도 진성 찐따나 트위터하는 페미찐따가 아니면 굳이 찾아가진 않으니까 근데 제일 무서운 찐따가 여자찐따인게 문제지 구하라 설리만 봐도 
어떻게 저얼굴을 지적해 니들아빠가 원빈급이면 조금이해는해볼게 사진좀 멀해도 예쁜승호 상처받지않길 
유승호에게 외모악플이란 이재용 재산걱정해주는것과 같음
세상에 너무 말라서 좀 쪘으면 했는데 살쪘다라니 유승호 배우님 언제나 응원합니다 잘생겨서 좋더만 무슨 
난 지금 모습 좋은데 원래 아역배우들 슬럼프 못이기는데 유승호는 아역배우 이미지 벗은것 같아 좋아요 그전에 했던 영화도 괜찮았고요 난 봉이 김선달부터 그사람의 배역에 충실함 지금은 다 좋은데 ㅎㅎ
요즘 경찰 방송쪽에 돈 엄청 쏟는가보네 무슨 경찰 소재 드라마가 이리 막 쏟아지냐 
유승호가 아무리 쪄도 악플 단 새끼 니보단 훨씬 잘생겼다
유승호 넘조타 하악 
 대의 한국 남자들은 유승호에게 관심도 없거니와 악플 다는 것조차 귀찮아 악플을 누가 달았겠어 그분들이지 쿵쾅쿵쾅
진짜 제목만봐도 코웃음난다 유승호가 얼굴로 까이다닠ㅋㅋㅋ
유승호 ㅐ대 배우중에 외모 탑급인데ㅋㅋㅋ
ㅋㅋㅋ 유승호는 건들지말자
한녀들 왜저래 적당히좀 해라
이놈의 악플러드 전부 벌금형이상으로 형량을높여야 없어지려나 
유승호한테 외모지적이라니 ㅋㅋㅋ ㅁㅊㄴ


In [67]:
import pandas as pd
from konlpy.tag import Okt
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

def train(comments, model_result):
    comment_data = pd.DataFrame(comments)
    comment_data.columns = ['comment']
    in_result = np.insert(model_result, 0, 0)
    de_result = np.delete(in_result,0)
    de_data = pd.DataFrame(de_result)
    de_data.columns = ['label']
    df_new = pd.concat([de_data, comment_data], axis = 1)
    X_data = df_new['comment']
    y_data = df_new['label']
    normalized_text = []
    for string in X_data.tolist():
        try:
            tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ", string.lower())
        except Exception as e:
            break
    normalized_text.append(tokens)
    #df_new.comment = normalized_text
    okt = Okt()
    X_token=[]
    for sentence in df_new['comment']:
        temp_X = []
        temp_X = okt.morphs(sentence, stem=True) # 토큰화
        X_token.append(temp_X)
    max_words = 10000
    tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
    tokenizer.fit_on_texts(X_token) 
    X_token = tokenizer.texts_to_sequences(X_token)
    word_to_index = tokenizer.word_index
    vocab_size = len(word_to_index) + 1
    max_len = 124
    X_data = pad_sequences(X_token, maxlen=max_len)
    y_data = np.array(y_data).reshape(-1, 1)
    model = load_model('model8n.h5')
    history = model.fit(X_data, y_data, epochs=5, batch_size=32)
    return history

In [68]:
train(s, model_result)

Train on 20 samples
Epoch 1/5
20/20 [==============================] - 2s 116ms/sample - loss: 0.1494 - acc: 1.0000
Epoch 2/5
20/20 [==============================] - 0s 14ms/sample - loss: 0.1378 - acc: 0.9500
Epoch 3/5
20/20 [==============================] - 0s 9ms/sample - loss: 0.0746 - acc: 1.0000
Epoch 4/5
20/20 [==============================] - 0s 11ms/sample - loss: 0.0213 - acc: 1.0000
Epoch 5/5
20/20 [==============================] - 0s 8ms/sample - loss: 0.0155 - acc: 1.0000


In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import re

In [ ]:
def get_text(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = ''
    for item in soup.find_all('div', id='articeBody'):
        text = text + str(item.find_all(text=True))
    return text

In [ ]:
txt=get_text('https://entertain.naver.com/read?oid=109&aid=0004166734')

In [ ]:
def clean_text(text):
    cleaned_text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\.]+", " ", text)
    return cleaned_text

In [ ]:
clean = clean_text(txt)

In [ ]:
def get_title(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = soup.select_one('h2.end_tit')
    text_r = text.get_text()
    return text_r
title = get_title('https://entertain.naver.com/read?oid=109&aid=0004166734')
title

In [ ]:
clean_text(title)

In [ ]:
from krwordrank.sentence import summarize_with_sentences
def keyword_sentence(url):#키워드 문장 추출
    m_text = get_text('https://entertain.naver.com/read?oid=109&aid=0004166734')
    r_text = clean_text(m_text)
    data = r_text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=11, num_keysents=10)
    keyword = list(keywords.keys())
    return keyword[0], sents

In [ ]:
keyword, sents = keyword_sentence(clean)
sents

In [ ]:
keyword

In [ ]:
def sentence_generation(current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    global keyword
    model = load_model('model_create.h5')
    max_words = 35000
    t = Tokenizer(num_words = max_words)
    init_word = current_word 
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=30, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for k, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + k # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + k # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
def clean_sentence(word):
    global keyword
    max_words = 35000
    clean_sen = []
    for i in range(0,10):
        sentence = sentence_generation(word, i)
        clean_sen.append(sentence)
        i += 1
    return clean_sen[9]

In [ ]:
clean_sentence(keyword)

In [ ]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    global word
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=30, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)# 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [ ]:
def clean_sentence(word):
    clean_sen = []
    model = load_model('model_create.h5')
    max_words = 35000
    t = Tokenizer(num_words = max_words)
    for i in range(0,10):
        sentence = sentence_generation(model, t, word, i)
        clean_sen.append(sentence)
        i += 1
    return clean_sen[9]

In [ ]:
clean_sentence(keyword)